In [1]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import numpy as np

In [2]:
def angle(A,B,C):
    import numpy as np
    A=np.array(A)
    B=np.array(B)
    C=np.array(C)

    BA=A-B
    BC=C-B
    norm_BA=np.linalg.norm(BA)
    norm_BC=np.linalg.norm(BC)
    if norm_BA < 1e-6 or norm_BC < 1e-6:
        return 0.0
    cosine=np.dot(BA,BC)/(norm_BA*norm_BC)
    cosine=np.clip(cosine,-1.0,1.0)
    cosine_angle=np.degrees(np.arccos(cosine))
    return int(cosine_angle)

In [3]:
def shoulder2wrist(landmark):
    s=landmark[11]
    e=landmark[13]
    w=landmark[15]

    shoulder=s.x,s.y
    elbow=e.x,e.y
    wrist=w.x,w.y
    shoulder2wrist_angle=angle(shoulder,elbow,wrist)
    return int(shoulder2wrist_angle)
    

In [4]:
def shoulder2knee(landmark):
    s=landmark[11]
    h=landmark[23]
    k=landmark[25]

    shoulder=s.x,s.y
    hip=h.x,h.y
    knee=k.x,k.y
    shoulder2knee_angle=angle(shoulder,hip,knee)
    return int(shoulder2knee_angle)

In [5]:
def elbow2hip(landmark):
    e=landmark[13]
    s=landmark[11]
    h=landmark[23]

    elbow=e.x,e.y
    shoulder=s.x,s.y
    hip=h.x,h.y
    elbow2hip_angle=angle(elbow,shoulder,hip)
    return int(elbow2hip_angle)

In [6]:
def hip2leg(landmark):
    h=landmark[23]
    k=landmark[25]
    a=landmark[27]
    
    hip=h.x,h.y
    knee=k.x,k.y
    ankle=a.x,a.y
    hip2leg_angle=angle(hip,knee,ankle)
    return int(hip2leg_angle)

In [7]:
"""pushup_direction='up'
pushup_count=0
def pushup(landmark):
    global pushup_direction,pushup_count
    sw=shoulder2wrist(landmark)
    sk=shoulder2knee(landmark)
    pushup_top={'cond1':170<=sw<=180, 'cond2':170<=sk>=180,'pushup_direction':'up'}
    pushup_bottom={'cond1':70<=sw<=90, 'cond2':170<=sk>=180,'pushup_direction':'down'}
    
    if 165 <= sw <= 185 and 165 <= sk <= 185:
        if pushup_direction == 'down':
            pushup_direction = 'up'
            pushup_count += 1

    # Bottom position (elbows bent)
    if 60 <= sw <= 100 and 165 <= sk <= 185:
        pushup_direction = 'down'

    return pushup_count"""

"pushup_direction='up'\npushup_count=0\ndef pushup(landmark):\n    global pushup_direction,pushup_count\n    sw=shoulder2wrist(landmark)\n    sk=shoulder2knee(landmark)\n    pushup_top={'cond1':170<=sw<=180, 'cond2':170<=sk>=180,'pushup_direction':'up'}\n    pushup_bottom={'cond1':70<=sw<=90, 'cond2':170<=sk>=180,'pushup_direction':'down'}\n    \n    if 165 <= sw <= 185 and 165 <= sk <= 185:\n        if pushup_direction == 'down':\n            pushup_direction = 'up'\n            pushup_count += 1\n\n    # Bottom position (elbows bent)\n    if 60 <= sw <= 100 and 165 <= sk <= 185:\n        pushup_direction = 'down'\n\n    return pushup_count"

In [8]:
pushup_direction = 'up'
pushup_count = 0

def pushup(landmark):
    global pushup_direction, pushup_count

    sw = shoulder2wrist(landmark)
    sk = shoulder2knee(landmark)

    # ---- TOP: Arms straight ----
    top = (165 <= sw <= 185) and (165 <= sk <= 185)

    # ---- BOTTOM: Elbows bent ----
    bottom = (60 <= sw <= 100) and (165 <= sk <= 185)

    # If bottom reached → direction = down
    if bottom:
        pushup_direction = 'down'

    # If top reached AND previous direction was down → rep++
    if top and pushup_direction == 'down':
        pushup_direction = 'up'
        pushup_count += 1

    return pushup_count


In [9]:
pullup_count = 0
pullup_direction = "down" 

def pullup(landmark):
    global pullup_count,pullup_direction
    sw=shoulder2wrist(landmark)
    if sw<=80 and pullup_direction == "down":
        pullup_direction = "up"
        pullup_count += 1
    elif sw>=150:
        pullup_direction ="down" 
    return pullup_count
        

In [10]:
"""squat_count=0
squat_direction='up'
def squats(landmark):
    global squat_count,squat_direction
    sq=hip2leg(landmark)
    if sq<=90 and squat_direction=='up':
        squat_count+=1
        squat_direction='down'
    elif 160<=sq<=180 and squat_direction=='down':
        squat_direction='up'
    return squat_count"""
    

"squat_count=0\nsquat_direction='up'\ndef squats(landmark):\n    global squat_count,squat_direction\n    sq=hip2leg(landmark)\n    if sq<=90 and squat_direction=='up':\n        squat_count+=1\n        squat_direction='down'\n    elif 160<=sq<=180 and squat_direction=='down':\n        squat_direction='up'\n    return squat_count"

In [11]:
squat_count = 0
squat_direction = 'up'

def squats(landmark):
    global squat_count, squat_direction

    sq = hip2leg(landmark)

    # ---- BOTTOM POSITION ----
    bottom = sq <= 90

    # ---- TOP POSITION ----
    top = 160 <= sq <= 185

    # If bottom reached → go down
    if bottom and squat_direction == 'up':
        squat_direction = 'down'

    # If top reached AND previous direction was down → count++
    if top and squat_direction == 'down':
        squat_direction = 'up'
        squat_count += 1

    return squat_count


In [12]:
squat_count = 0
squat_direction = 'up'

def squats(landmark):
    global squat_count, squat_direction

    # Compute knee angle
    sq = hip2leg(landmark)

    # Define thresholds
    BOTTOM_THRESHOLD = 95       # squatting depth
    TOP_THRESHOLD = 160         # standing straight

    bottom = sq <= BOTTOM_THRESHOLD
    top = sq >= TOP_THRESHOLD

    # ----- Detect going down -----
    if bottom and squat_direction == 'up':
        squat_direction = 'down'

    # ----- Detect coming back up (count rep) -----
    if top and squat_direction == 'down':
        squat_direction = 'up'
        squat_count += 1

    return squat_count


In [13]:
def put_text(frame, text, x, y):
    cv2.putText(frame, text, (x, y),
                cv2.FONT_HERSHEY_SIMPLEX,
                1.0, (255, 0, 0), 2, cv2.LINE_AA)


In [14]:
latest_frame=None
from mediapipe.python.solutions.pose import POSE_CONNECTIONS
workout=str(input("What Workout bro?"))

def callback(result,output_image,timestamp_ms):
    global latest_frame,workout
    global pushup_direction,pushup_count
    global pullup_count,pullup_direction
    global squat_count,squat_direction
    
    frame=output_image.numpy_view()
    frame=cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
    if not result.pose_landmarks:
        latest_frame = frame
        return
    landmark=result.pose_landmarks[0]
    h,w=frame.shape[:2]
    for l in landmark:
        cx,cy=int(l.x*w),int(l.y*h)
        cv2.circle(frame,(cx,cy),5,(0,0,255),-1)

    
    for start,end in POSE_CONNECTIONS:
        x1,y1=landmark[start].x,landmark[start].y
        x2,y2=landmark[end].x,landmark[end].y
        x1,y1=int(x1*w),int(y1*h)
        x2,y2=int(x2*w),int(y2*h)
        cv2.line(frame,(x1,y1),(x2,y2),(0,255,0),3)

    latest_frame = frame

    
    if workout=='pushup':
        pushup_count=pushup(landmark)
        put_text(frame, f"Pushups: {pushup_count}", 50, 80)

    elif workout=='pullup':
        pullup_count=pullup(landmark)
        put_text(frame, f"Pullups: {pullup_count}", 50, 80)

    elif workout=='squat':
        squat_count=squats(landmark)
        put_text(frame, f"Squats: {squat_count}", 50, 80)

What Workout bro? squat


In [15]:
path=r"C:\Hope AI\course\Deep Learning\DL Project\pose_landmarker_heavy.task"
base=python.BaseOptions(model_asset_path=path)
options=vision.PoseLandmarkerOptions(base_options=base,running_mode=vision.RunningMode.LIVE_STREAM,result_callback=callback)
Landmark=vision.PoseLandmarker.create_from_options(options)

In [16]:
cap=cv2.VideoCapture(0)
mp_image_format = mp.ImageFormat
import time
while True:
    ret,frame=cap.read()
    if not ret:
        break

    h,w=frame.shape[:2]
    timestamp_ms=int(time.time() * 1000)
    rgb_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    mp_img=mp.Image(image_format=mp_image_format.SRGB,data=rgb_frame)

    result=Landmark.detect_async(mp_img,timestamp_ms)
    if latest_frame is not None:
        display=latest_frame
        #cv2.imshow("Latest Frame",latest_frame)
    else:
        display=frame
        #cv2.imshow("frame",frame)
    cv2.imshow("Pose Stream",display)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

#print(pushup_count)

In [17]:
print(pushup_count)
print(pullup_count)

0
0


In [18]:
squat_count

5